In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
batch_size = 16 
block_size = 32 
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [3]:
torch.manual_seed(1337)

In [4]:
with open('ประมวลกฎหมายอาญา.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(''.join(chars))
print(vocab_size)


 ()-./:[]กขคฆงจฉชซญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤลวศษสหอฯะัาำิีึืุูเแโใไๆ็่้๋์๐๑๒๓๔๕๖๗๘๙“”
84


In [6]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
pre_decode = lambda l: [itos[i] for i in l]
decode = lambda l: ''.join([itos[i] for i in l])

print(stoi)
print('---')
print(itos)
print('---')
print(encode('กฎหมาย'))
print(pre_decode(encode('กฎหมาย')))
print(decode(encode('กฎหมาย')))

{'\n': 0, ' ': 1, '(': 2, ')': 3, '-': 4, '.': 5, '/': 6, ':': 7, '[': 8, ']': 9, 'ก': 10, 'ข': 11, 'ค': 12, 'ฆ': 13, 'ง': 14, 'จ': 15, 'ฉ': 16, 'ช': 17, 'ซ': 18, 'ญ': 19, 'ฎ': 20, 'ฏ': 21, 'ฐ': 22, 'ฑ': 23, 'ฒ': 24, 'ณ': 25, 'ด': 26, 'ต': 27, 'ถ': 28, 'ท': 29, 'ธ': 30, 'น': 31, 'บ': 32, 'ป': 33, 'ผ': 34, 'ฝ': 35, 'พ': 36, 'ฟ': 37, 'ภ': 38, 'ม': 39, 'ย': 40, 'ร': 41, 'ฤ': 42, 'ล': 43, 'ว': 44, 'ศ': 45, 'ษ': 46, 'ส': 47, 'ห': 48, 'อ': 49, 'ฯ': 50, 'ะ': 51, 'ั': 52, 'า': 53, 'ำ': 54, 'ิ': 55, 'ี': 56, 'ึ': 57, 'ื': 58, 'ุ': 59, 'ู': 60, 'เ': 61, 'แ': 62, 'โ': 63, 'ใ': 64, 'ไ': 65, 'ๆ': 66, '็': 67, '่': 68, '้': 69, '๋': 70, '์': 71, '๐': 72, '๑': 73, '๒': 74, '๓': 75, '๔': 76, '๕': 77, '๖': 78, '๗': 79, '๘': 80, '๙': 81, '“': 82, '”': 83}
---
{0: '\n', 1: ' ', 2: '(', 3: ')', 4: '-', 5: '.', 6: '/', 7: ':', 8: '[', 9: ']', 10: 'ก', 11: 'ข', 12: 'ค', 13: 'ฆ', 14: 'ง', 15: 'จ', 16: 'ฉ', 17: 'ช', 18: 'ซ', 19: 'ญ', 20: 'ฎ', 21: 'ฏ', 22: 'ฐ', 23: 'ฑ', 24: 'ฒ', 25: 'ณ', 26: 'ด', 27: 'ต', 28: 

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 
        
        wei = q @ k.transpose(-2,-1) * C**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

    
        tok_emb = self.token_embedding_table(idx) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x)
        logits = self.lm_head(x) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
          
            idx_cond = idx[:, -block_size:]
   
            logits, loss = self(idx_cond)
           
            logits = logits[:, -1, :] 
         
            probs = F.softmax(logits, dim=-1) 
           
            idx_next = torch.multinomial(probs, num_samples=1)
         
            idx = torch.cat((idx, idx_next), dim=1) 
        return idx

model = BigramLanguageModel()
m = model.to(device)

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.21218 M parameters


In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

  
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")


    xb, yb = get_batch('train')


    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 4.5728, val loss 4.5425
step 100: train loss 2.8638, val loss 3.0167
step 200: train loss 2.5459, val loss 2.5134
step 300: train loss 2.2607, val loss 2.2267
step 400: train loss 2.0289, val loss 1.9973
step 500: train loss 1.8656, val loss 1.8677
step 600: train loss 1.7330, val loss 1.7938
step 700: train loss 1.5998, val loss 1.7048
step 800: train loss 1.5563, val loss 1.6997
step 900: train loss 1.4753, val loss 1.6726
step 1000: train loss 1.4475, val loss 1.6490
step 1100: train loss 1.3849, val loss 1.6122
step 1200: train loss 1.3339, val loss 1.6134
step 1300: train loss 1.3252, val loss 1.6266
step 1400: train loss 1.2723, val loss 1.6031
step 1500: train loss 1.2307, val loss 1.6189
step 1600: train loss 1.2287, val loss 1.6066
step 1700: train loss 1.2133, val loss 1.5759
step 1800: train loss 1.1878, val loss 1.5558
step 1900: train loss 1.1757, val loss 1.5331
step 2000: train loss 1.1712, val loss 1.5678
step 2100: train loss 1.1241, val loss 1.5457
